### Funktion um Daten per Id zu laden 

In [4]:
import requests, json

def get_data_by_id(id):
    url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/rki_key_data_v/FeatureServer/0/query?"
    
    parameter = {
        'referer':'https://www.mywebapp.com',
        'user-agent':'python-requests/2.9.1',
        'where': f'AdmUnitId = {id}', # Welche landkreise sollen zurück gegeben werden
        'outFields': '*', # Rückgabe aller Felder
        'returnGeometry': False, # Keine Geometrien
        'f':'json', # Rückgabeformat, hier JSON
        'cacheHint': True # Zugriff über CDN anfragen
    }
    result = requests.get(url=url, params=parameter) #Anfrage absetzen
    return json.loads(result.text) # Das Ergebnis JSON als Python Dictionary laden

### Die 10 größten Städte Deutschlands mit AdminUnitId

In [5]:
import pandas as pd

# als dictionary
große_staedte= {'AdminUnitId': {
  'Berlin': 11,
  'München': 9162,
  'Hamburg': 2000,
  'Köln': 5315,
  'Frankfurt': 6412,
  'Stuttgart': 8111,
  'Düsseldurf': 5111,
  'Dortmund': 5913,
  'Essen': 5113, }}

# umwandlung in dataframe
df_gr_staedte= pd.DataFrame(große_staedte)
df_gr_staedte

,AdminUnitId
Berlin,11
Dortmund,5913
Düsseldurf,5111
Essen,5113
Frankfurt,6412
Hamburg,2000
Köln,5315
München,9162
Stuttgart,8111


### Datensätze des RKI für die 10 größten Städte


In [6]:
berlin_data= get_data_by_id(11)
dortmund_data= get_data_by_id(5913)
düsseldorf_data= get_data_by_id(5111)
essen_data= get_data_by_id(5113)
frankfurt_data= get_data_by_id(6412)
hamburg_data= get_data_by_id(2000)
köln_data= get_data_by_id(5315)
münchen_data= get_data_by_id(9162)
stuttgart_data= get_data_by_id(6412)

Quelle für AdminUnitId: https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0/explore

### Datensatz RKI  - hier Berlin

In [7]:
dict_berlin = berlin_data['features'][0]  # berlin_data in dict umwandeln
df_berlin_data= pd.DataFrame(dict_berlin) # in dataframe umwandeln

df_berlin_data

,attributes
AdmUnitId,11.0
AnzAktiv,10700.0
AnzAktivNeu,800.0
AnzFall,225044.0
AnzFall7T,5719.0
AnzFallNeu,1498.0
AnzGenesen,210600.0
AnzGenesenNeu,700.0
AnzTodesfall,3709.0
AnzTodesfallNeu,5.0


In [8]:
# zum ausprobieren für mich
df_berlin_data.index[3]

'AnzFall'

In [9]:
# zum ausprobieren für mich
df_berlin_data['attributes']['AnzFall']

225044.0

### Definition um nur einzelne Daten zu extrahieren

In [19]:
def extract_info(dictionary, cityname):
    fallzahl = dictionary['attributes']['AnzFall']
    inzidenz= dictionary['attributes']['Inz7T']
    fallzahl_7_tage= dictionary['attributes']['AnzFall7T']
    new_dict= {}
    new_dict.update(Fallzahl = fallzahl, Inzidenz_7_Tage = inzidenz, Fallzahl_7_tage = fallzahl_7_tage )
    dict_final = {}
    dict_final [cityname] = new_dict 
   
    return pd.DataFrame(dict_final) # Ausgabe als DataFrame - Zweck: Ansicht als Tabelle
    

In [20]:
berlin = extract_info(dict_berlin, "Berlin")
berlin

,Berlin
Fallzahl,225044.0
Fallzahl_7_tage,5719.0
Inzidenz_7_Tage,156.1


### Städte im Vergleich

In [17]:
# Hamburg
dict_hamburg = hamburg_data['features'][0]  # hamburg_data in dict umwandeln

hamburg = extract_info(dict_hamburg, "Hamburg")

hamburg

,Hamburg
Fallzahl,99067.0
Fallzahl_7_tage,2134.0
Inzidenz_7_Tage,115.2


In [18]:
berlin = extract_info(dict_berlin, "Berlin")

berlin

,Berlin
Fallzahl,225044.0
Fallzahl_7_tage,5719.0
Inzidenz_7_Tage,156.1
